In [6]:
from torch.utils.data import DataLoader
import torchvision
import torch
from tqdm import tqdm
import pandas as pd
import os
import pydicom
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn

from utils import normalize_dicom

In [7]:
df_st = pd.read_csv("data/v2/study.csv")
df_st = df_st.set_index("ID")
df_st = df_st.dropna(subset="DOMINANCE")

df_im = pd.read_csv("data/v2/image.csv", sep=";")
df_im = df_im.set_index("ID")
df_if = pd.read_csv("data/v2/imagefeedback.csv", sep=";")
df_if = df_if.set_index("IMAGE_ID")

In [8]:
dcm_files = os.listdir("data/frame_selection/dicoms/")
dcm_files = [f for f in dcm_files if f.endswith(".dcm")]

parent = "data/frame_selection/dicoms/"
images = {}
for filename in tqdm(dcm_files):
    dcm = pydicom.dcmread(parent + filename)
    img = normalize_dicom(dcm)
    images[filename.replace(".dcm", "")] = img

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [02:15<00:00,  3.99it/s]


In [9]:
studies = df_st[["DOMINANCE"]]
imgs = df_im.join(studies, how="inner", on="STUDY_ID")
imgs = imgs.join(df_if)
imgs = imgs.dropna(subset="SIDE")
imgs = imgs.loc[imgs.index.map(lambda x: x in images)]
imgs = imgs.join(imgs.groupby("STUDY_ID")["SIDE"].mean(), on="STUDY_ID", rsuffix="_mean")
imgs = imgs.loc[(imgs["SIDE_mean"]>0) & (imgs["SIDE_mean"]<1)]
imgs

,ACQUISITIONDATE,ACQUISITIONNO,ACQUISITIONUID,AGE,BATCH,BODYPARTEXAMINED,DATASET,FRAMERATE,FRAMES,INSTANCENO,...,DOMINANCE,ID,CREATEDBY,CREATEDON,SEGMENTATION_ID,DONE,studyFeedback_id,frame,SIDE,SIDE_mean
ID,,,,,,,,,,,,,,,,,,,,,
131aedfhs6pnf1fvtvp493iu1n6d1qnz22,NaN,NaN,NaN,NaN,34,NaN,Rzeszow-MSW2,0,76,NaN,...,1.0,2a1b15a0-39cf-4893-8ed0-a2246f474411,sgurba@gmail.com,2023-12-30 21:45,2a1b15a0-39cf-4893-8ed0-a2246f474411,0.0,ce8a09b0-70e3-4c4a-a711-a3e8fadf1694,34.0,0.0,0.5
131aedfhs6pnf1fvtvp493iu1n8gd3gs22,NaN,NaN,NaN,NaN,34,NaN,Rzeszow-MSW2,0,70,NaN,...,1.0,2e35aa91-5c1e-4856-a270-db5476db36dc,sgurba@gmail.com,2023-12-30 21:45,2e35aa91-5c1e-4856-a270-db5476db36dc,0.0,ce8a09b0-70e3-4c4a-a711-a3e8fadf1694,31.0,0.0,0.5
131aedfhs6pnf1fvtvp493iu1na8zz7s22,NaN,NaN,NaN,NaN,34,NaN,Rzeszow-MSW2,0,59,NaN,...,1.0,f1dad2cb-9cb4-45c6-97c2-557416b192c6,sgurba@gmail.com,2023-12-30 21:45,f1dad2cb-9cb4-45c6-97c2-557416b192c6,0.0,ce8a09b0-70e3-4c4a-a711-a3e8fadf1694,22.0,0.0,0.5
131aedfhs6pnf1fvtvp493iu1nekk6h022,NaN,NaN,NaN,NaN,34,NaN,Rzeszow-MSW2,0,43,NaN,...,1.0,31385931-a25b-4745-b88b-38c5fd058bdd,sgurba@gmail.com,2023-12-30 21:45,31385931-a25b-4745-b88b-38c5fd058bdd,0.0,ce8a09b0-70e3-4c4a-a711-a3e8fadf1694,18.0,1.0,0.5
131aedfhs6pnf1fvtvp493iu1neqswr222,NaN,NaN,NaN,NaN,34,NaN,Rzeszow-MSW2,0,61,NaN,...,1.0,4a61b044-3967-46fa-b240-872af197f698,sgurba@gmail.com,2023-12-30 21:45,4a61b044-3967-46fa-b240-872af197f698,0.0,ce8a09b0-70e3-4c4a-a711-a3e8fadf1694,32.0,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13c2ur549vohc0jat2wqi9spx2,20180213.0,9.0,NaN,076Y,7,NaN,data,0,37,NaN,...,1.0,2560349c-d557-4540-9e38-dfc1ff9ef792,sgurba@gmail.com,2023-12-23 13:23,2560349c-d557-4540-9e38-dfc1ff9ef792,0.0,57ca094f-50be-4a16-a08c-026362ceb2eb,19.0,0.0,0.4
13c2ur549vohc0jat2wqi9syl1,20180213.0,10.0,NaN,076Y,7,NaN,data,0,19,NaN,...,1.0,959a1a20-7e85-453a-86e1-e057153a5ff0,sgurba@gmail.com,2023-12-23 13:23,959a1a20-7e85-453a-86e1-e057153a5ff0,0.0,57ca094f-50be-4a16-a08c-026362ceb2eb,11.0,0.0,0.4
13c2ur549vohc0jat2wqi9t2i1,20180213.0,13.0,NaN,076Y,7,NaN,data,0,31,NaN,...,1.0,35aa568b-8249-4675-ad44-0cf741ea9c26,sgurba@gmail.com,2023-12-23 13:23,35aa568b-8249-4675-ad44-0cf741ea9c26,0.0,57ca094f-50be-4a16-a08c-026362ceb2eb,20.0,0.0,0.4


In [10]:
test_study_ids = pd.read_csv("teststudies.csv")
test_study_ids = set(test_study_ids.ID)

In [11]:
class DominanceDataset:  
    def __init__(self, images: dict, df: pd.DataFrame, split=None):
        self.images = images
        self.studies = df["STUDY_ID"].unique()
        self.df = df

        if split == "train":
            self.studies = [s for s in self.studies if s not in test_study_ids]
        elif split == "test":
            self.studies = [s for s in self.studies if s in test_study_ids]
        
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224), antialias=None),
            transforms.Normalize([0.5], [0.5])   
        ])
                
    def __len__(self):
        return len(self.studies)

    def __getitem__(self, idx):
        study_id = self.studies[idx]
        img_subset = self.df[self.df["STUDY_ID"]==study_id]
        label = img_subset.iloc[0]["DOMINANCE"]
        
        left = img_subset[img_subset.SIDE == 0].sample(1).reset_index(names="idx").iloc[0]
        right = img_subset[img_subset.SIDE == 1].sample(1).reset_index(names="idx").iloc[0]

        img_left = self.images[left.idx][int(left.frame)]
        img_right = self.images[right.idx][int(right.frame)]
        img_left = self.transform(img_left)
        img_right = self.transform(img_right)
        
        return img_left, img_right, label
        

In [16]:
class SiameseResNet(torch.nn.Module):
    def __init__(self):
        super(SiameseResNet, self).__init__()
        self.base_model = torchvision.models.resnet18(pretrained=True)
        self.base_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.base_model.fc = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(512 * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
        )

    def forward(self, img1, img2):
        features1 = self.base_model(img1)
        features2 = self.base_model(img2)
        combined_features = torch.cat((features1, features2), dim=1)
        result = self.classifier(combined_features)
        return result

train_dataset = DominanceDataset(images, imgs, split="train")
test_dataset = DominanceDataset(images, imgs, split="test")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SiameseResNet()
model = model.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_test_loss = np.inf
pbar = tqdm(range(100))
for epoch in pbar:
    model.train()
    running_loss = 0.0
    correct = 0
    for imgs_left, imgs_right, labels in train_loader:
        imgs_left = imgs_left.to(device)
        imgs_right = imgs_right.to(device)
        labels = labels.to(device).float().unsqueeze(1) 

        outputs = model(imgs_left, imgs_right)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += ((outputs > 0.5).float() == labels).sum().item()

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = correct / len(train_dataset) * 100

    model.eval()
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        for imgs_left, imgs_right, labels in test_loader:
            imgs_left = imgs_left.to(device)
            imgs_right = imgs_right.to(device)
            labels = labels.to(device).float().unsqueeze(1) 
    
            outputs = model(imgs_left, imgs_right)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            correct += ((outputs > 0.5).float() == labels).sum().item()

    avg_test_loss = running_loss / len(test_loader)
    test_accuracy = correct / len(test_dataset) * 100

    pbar.set_description(
        f'Loss: {avg_train_loss:.4f}/{avg_test_loss:.4f} | ' 
        f'Acc {train_accuracy:.2f}/{test_accuracy:.2f}'
    )

    if avg_test_loss < best_test_loss:
        best_test_loss = avg_test_loss
        torch.save(model.state_dict(), 'dominance.pth')


/root/przemek/miniconda3/envs/torch21/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/przemek/miniconda3/envs/torch21/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loss: 0.0259/0.6987 | Acc 100.00/73.33: 100%|██████████████████████████████████████████████████████████████████████████████████| 100/100 [01:49<00:00,  1.09s/it]
